In [1]:
!pip install transformers

     |████████████████████████████████| 890kB 5.3MB/s 
     |████████████████████████████████| 1.1MB 19.7MB/s 
     |████████████████████████████████| 3.0MB 39.5MB/s 
     |████████████████████████████████| 890kB 42.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=6216e003cc7508ed936828a3d5854079754cd8471d3078d708c5d0bb37057a8d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import torch

In [3]:
## Import the BERT for Question and Answer Model . This class fine tunes the model for benchmark data
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [4]:
## Import the BERTTokenizer
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [5]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')

In [8]:
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

bert_abstract = "Unity Connection supports HTTPS Networking, that allows you to connect different Unity Connection servers and clusters in a single site network. HTTP networking provides more scalable Unity Connection deployments as compared to legacy networking. The architecture of HTTPS networking is scalable both in terms of number of Unity Connection locations and the total directory size. HTTPS Protocol is used for directory synchronization within a network.In addition to HTTPS networking, Unity Connection also supports legacy networking to connect multiple Unity Connection servers in a network. However, you should deploy a new network as per HTTPS networking. Legacy networking includes both intrasite (digital) and intersite networking.The legacy and HTTPS networking are not supported simultaneously in the same network. In legacy networking, SMTP is the method used within a site, and HTTPS is used in Intersite networking when linking two separate sites."

print(wrapper.fill(bert_abstract))

Unity Connection supports HTTPS Networking, that allows you to connect different
Unity Connection servers and clusters in a single site network. HTTP networking
provides more scalable Unity Connection deployments as compared to legacy
networking. The architecture of HTTPS networking is scalable both in terms of
number of Unity Connection locations and the total directory size. HTTPS
Protocol is used for directory synchronization within a network.In addition to
HTTPS networking, Unity Connection also supports legacy networking to connect
multiple Unity Connection servers in a network. However, you should deploy a new
network as per HTTPS networking. Legacy networking includes both intrasite
(digital) and intersite networking.The legacy and HTTPS networking are not
supported simultaneously in the same network. In legacy networking, SMTP is the
method used within a site, and HTTPS is used in Intersite networking when
linking two separate sites.


In [11]:
question = "Hi How are you ?"

answer_question(question, bert_abstract)

Query has 178 tokens.

Answer: "deploy"
